In [1]:
# goal: use the mined dataset to create setting to verify a proposed similarity 
# measure between two outputs of platform A and platform B.

In [1]:
import numpy as np
import cirq
import qiskit
from qiskit.circuit import qpy_serialization
from qiskit import QuantumCircuit
from typing import List
import math
import pandas as pd
from IPython.display import display
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os

In [3]:
REAL_DATA_FOLDER = "../data/random_program_execution"
RANDOM_DATA_FOLDER = "../data/random_program_execution"

In [4]:
records = [
    {
        "filepath": f,
        "hash": f.split("_")[0],
        "platform": f.split("_")[1].replace(".csv", "")
    } 
    for f in os.listdir(REAL_DATA_FOLDER)
    if f.endswith(".csv")
]
ids = list(set([r['hash'] for r in records]))
print(f"We have {len(ids)} records.")
ids[:5]

We have 1000 records.


['c4dd50fff08a5e69c61338ab4e4940baf2e1ef5e40588e509279ca210703a24d',
 '8859661d0f5ffa5a7e800954c660190aaa1ec7b012f64a65e02b57a3af8f4119',
 '00b8044c61f32593c7ed45a95f80e64d954f22d349d70df34bf7cecccfa9651f',
 '7f3448d0431c205cbdadda5ac4dcaadb30d076d6808553efb93bd29471d63608',
 '98a2731f8b61cf516471aa31e78f8b4dcb0b32e56329208bcccda4538dcf3083']

In [5]:
records_random = [
    {
        "filepath": f,
        "random": f.replace(".csv", "")
    } 
    for f in os.listdir(RANDOM_DATA_FOLDER)
    if f.endswith(".csv")
]
ids_random = list(set([r['random'] for r in records_random]))

In [30]:
def create_pairs(ids_left, suffix_left, folder_left,
                 ids_right, suffix_right, folder_right,
                 n_samples, shared_ids=False):
    """Create pairs of experiments left and rights.
    
    shared_ids: boolean
        whether the sampling procedure should be done once and the IDs
        should be shared, aka we get the same circuit from the two platforms
    """
    left_candidates = np.random.choice(ids_left, size=n_samples, replace=False)
    right_candidates = np.random.choice(ids_right, size=n_samples, replace=False)
    if shared_ids:
        assert sorted(ids_right) == sorted(ids_left), "IDs left and right lists are not matching"
        right_candidates = left_candidates
    left_dfs = [
        pd.read_csv(
            os.path.join(folder_left, f"{id_left}{suffix_left}.csv")
        )
        for id_left in left_candidates
    ]
    right_dfs = [
        pd.read_csv(
            os.path.join(folder_right, f"{id_right}{suffix_right}.csv")
        )
        for id_right in right_candidates
    ]
    return zip(left_dfs, right_dfs)
    

In [7]:
pairs = create_pairs(ids_left=ids, suffix_left="_qiskit", folder_left=REAL_DATA_FOLDER,
                     ids_right=ids_random, suffix_right="", folder_right=RANDOM_DATA_FOLDER,
                     n_samples=50)

In [8]:
from IPython.display import display, HTML

In [9]:
for df_left, df_right in pairs:
    display(df_left.head())
    display(df_right.head())
    break

,Unnamed: 0,0100101001,0001001010,1010001010,1000111000,0110111110,1101011001,0111011110,1000101011,1001010111,...,0100011110,0100000100,0110100000,1000101111,0110111001,0111100101,1111000011,1110101001,1011111011,0111001001
0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,1.0,2.0,2.0,NaN,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,3.0,1.0,1.0,1.0,NaN,3.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,3.0,NaN,1.0,2.0,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,2.0,4.0,NaN,3.0,NaN,2.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,0111001100,1000111010,1000011010,0101001101,0111101101,0111111001,0111001001,1000011100,0011010101,...,0010101001,0110010000,1001000000,1111011101,1011000100,0100010001,0000001110,1011110101,0100000100,1011010100
0,0,15,33,21,23,13,2.0,10,10,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,17,35,24,13,19,3.0,9,10,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24,32,26,18,18,1.0,13,6,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,20,47,17,20,12,3.0,12,4,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,19,38,20,17,14,1.0,9,8,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Similarity Measures

In [37]:
from scipy.stats import ttest_ind

TOP_K_SOLUTION_TO_COMPARE = 5

def score_top_k(df_master, df_slave, first_k=TOP_K_SOLUTION_TO_COMPARE):
    # get the top k occurring solutions according to the master
    m = df_master.fillna(0).to_numpy()
    solution_cum_freq = m.sum(axis=0)
    most_frequent_solutions = np.argsort(solution_cum_freq)[::-1][:first_k]
    relevant_solutions = df_master.columns[most_frequent_solutions]
    freqs = solution_cum_freq[most_frequent_solutions]
    # compute p-values
    p_values = []
    df_master = df_master.fillna(0)
    df_slave = df_slave.fillna(0)
    for c in relevant_solutions:
        try:
            stat, p = ttest_ind(df_master[c], df_slave[c])
        except:
            if c not in df_master.columns:
                print(f"{c} column was missing from result of platform master")
                df_master[c] = 0
            if c not in df_slave.columns:
                print(f"{c} column was missing from result of platform slave")
                df_slave[c] = 0
            stat, p = ttest_ind(df_master[c], df_slave[c])    
        p_values.append(p)
    return min(p_values)

# Interpretation Routine

In [27]:
def interpret_as_p_value(score):
    print(f"P-Value like interpretation for: {score}")
    if score > 0.05:
        print('Probably the same distribution')
    else:
        print('Probably different distributions')

# Method Selection

In [23]:
detector = score_top_k

# Distinguish between: Real Quantum vs Uniform Random 

In [28]:
pairs = create_pairs(ids_left=ids, suffix_left="_qiskit", folder_left=REAL_DATA_FOLDER,
                     ids_right=ids_random, suffix_right="", folder_right=RANDOM_DATA_FOLDER,
                     n_samples=50)
scores = []
for df_left, df_right in pairs:
    score = detector(df_left, df_right)
    scores.append(score)
average_score = np.mean(scores)
interpret_as_p_value(average_score)

0110110010 column was missing from result of platform slave
0010010101 column was missing from result of platform slave
1010001100 column was missing from result of platform slave
1100111000 column was missing from result of platform slave
1001100101 column was missing from result of platform slave
0000110100 column was missing from result of platform slave
1010111110 column was missing from result of platform slave
0010110010 column was missing from result of platform slave
0011000000 column was missing from result of platform slave
1101001011 column was missing from result of platform slave
1110001111 column was missing from result of platform slave
1110010111 column was missing from result of platform slave
1101011101 column was missing from result of platform slave
1001011010 column was missing from result of platform slave
0110101110 column was missing from result of platform slave
0001100001 column was missing from result of platform slave
1000011011 column was missing from resul

# Distinguish between: Same (artificial) Distribution (Random)

In [29]:
pairs = create_pairs(ids_left=ids_random[:500], suffix_left="", folder_left=RANDOM_DATA_FOLDER,
                     ids_right=ids_random[500:], suffix_right="", folder_right=RANDOM_DATA_FOLDER,
                     n_samples=50)
scores = []
for df_left, df_right in pairs:
    score = detector(df_left, df_right)
    scores.append(score)
average_score = np.mean(scores)
interpret_as_p_value(average_score)

0110000001 column was missing from result of platform slave
0110001001 column was missing from result of platform slave
0001111010 column was missing from result of platform slave
0101100011 column was missing from result of platform slave
0110101010 column was missing from result of platform slave
0101100100 column was missing from result of platform slave
1000111000 column was missing from result of platform slave
1001000000 column was missing from result of platform slave
0011111111 column was missing from result of platform slave
0000001110 column was missing from result of platform slave
0010001111 column was missing from result of platform slave
0011010110 column was missing from result of platform slave
0101010101 column was missing from result of platform slave
0101101000 column was missing from result of platform slave
1001110100 column was missing from result of platform slave
1100000100 column was missing from result of platform slave
1100100011 column was missing from resul

**OBSERVATION**: testing only the first top-k (aka 5) bitstrings suffers when the distributions are very spread (uniform distributions)

# Distinguish: Same Platform, Same Circuit
Assumption: there are no bug in the sampling procedure of one platform

In [40]:
pairs = create_pairs(ids_left=ids, suffix_left="_qiskit", folder_left=REAL_DATA_FOLDER,
                     ids_right=ids, suffix_right="_cirq", folder_right=REAL_DATA_FOLDER,
                     n_samples=50, shared_ids=True)
scores = []
for df_left, df_right in pairs:

    for df in [df_left, df_right]:
        # NOTE THAT WE COMPARE THE SAME PLATFORM  
        # use the first half sampling on one side and the second half on the other
        half = int(len(df) / 2)
        score = detector(df[:half], df[half:])
        scores.append(score)
    
print('---- overall ----')
average_score = np.mean(scores)
interpret_as_p_value(average_score)
print('---- platfrom specific ----')
scores_left = [s for i, s in enumerate(scores) if i % 2 == 0]
scores_right = [s for i, s in enumerate(scores) if i % 2 != 0]
for name, scores in [('left', scores_left), ('right', scores_right)]:
    print(f'Platform {name}:')
    average_score = np.mean(scores)
    interpret_as_p_value(average_score)

P-Value like interpretation for: 0.06567255596699031
Probably the same distribution
Platform left:
P-Value like interpretation for: 0.06502692517881836
Probably the same distribution
Platform right:
P-Value like interpretation for: 0.06631818675516228
Probably the same distribution
